### 1. Persiapan

#### a. Install Library

In [ ]:
!pip install -U bitsandbytes
!pip install -U peft
!pip install -U trl
!pip install -U accelerate
!pip install -U datasets
!pip install -U transformers
!pip install -U wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beh

#### b. Import Library

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)

import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format

#### c. Setup API Token HuggingFace dan Wandb

In [ ]:
from huggingface_hub import login

hf_token = # your token
wb_token = # your token

login(token=hf_token)
wandb.login(key=wb_token)

run = wandb.init(
    project='Fine-tune Gemma 2B on Medical ID Dataset',
    job_type="training",
    anonymous="allow"
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rafyardhanie (rafyardhaniee). Use `wandb login --relogin` to force relogin


### 2. Setup Model

#### a. QLoRA Config

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

base_model = "google/gemma-2b-it"

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map='auto'
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

#### b. Setup Tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model)
model, tokenizer = setup_chat_format(
    model, tokenizer
)

#### c. Test Inference

In [ ]:
messages = [
    {"role": "user", "content": "Dok antibiotik yg ampuh untuk radang tenggorokan apa ya? sudah seminggu ini adik saya usia 10 tahun mengalami radang tenggorokan dan amandel juga merah. sudah minum obat paracetamol tapi demamnya turun sebentar lalu naik lagi. antibiotik yg tepat untuk anak umur 10 tahun apa yang tepat untuk anak 10 tahun.. terima kasih"}
]

prompt = tokenizer.apply_chat_template(messages,
                                       tokenize=False,
                                       add_generation_prompt=True)
inputs = tokenizer(prompt,
                   return_tensors='pt',
                   padding=True,
                   truncation=True)

outputs = model.generate(**inputs, max_length=150, num_return_sequences=1)
text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



**

**Antibiotik yang Ampuh untuk Radang Tenggorokan:**

* **Amoxicillin**
* **Nalidixicillin**

**Catatan:**

* Pastikan anak-anak Anda tidak mengonsumsi makanan atau minuman yang mengandung bahan-baku seperti antibiotik.
* Gunakan secara ter


#### d. Parameter Efficient Fine-tuning (PEFT)

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

print(model)

peft_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules='all-linear',
    lora_dropout=0.05,
    bias='none',
    task_type='CASUAL_LM'
)

model = get_peft_model(model, peft_config)

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256002, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): G

#### e. Trainable Parameters

In [ ]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 78446592 | total: 2584623104 | Percentage: 3.0351%


### 3. Persiapan Dataset

#### a. Load Training Dataset

In [ ]:
dataset = load_dataset('hermanshid/doctor-id-qa')
dataset

README.md:   0%|          | 0.00/198 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/4.16M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/466k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5694 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/633 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 5694
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 633
    })
})

#### b. Melihat Beberapa Contoh Dataset

In [ ]:
print(dataset['train']['question'][0])
print(dataset['train']['answer'][0])

Assalamaualaikum dok, saya mau konsul mengenai feses bayi. Bayi saya baru berusia 3 bulan, semalam pup warna fesesnya kuning pucat. Padahal sore harinya saat pup fesesnya berwarna normal, bayi saya minum asui dan sufor dok. Kondisinya bayi saya tidak sedang demam, tidak rewel dan masih aktif. Apakah berbahaya dok?
Waalaikumsalm. warna fases bayi serta konsentrasinya bisa berubah-ubah selama beberapa hari, minggu dan bulan pertama setelah lahir. usia, pola makan dan kondisi kesehatan bisa mempengaruhi perubahan warna fases bayi, warna fasse bayi berwarna kuning atau coklat. ASI dan susu formula juga memperngaruhi warna fases bayi.warna fases bayi berwarna kuning merupakan warna fases yang normal, terutama pada bayi yang di susui.


#### c. Memilih Dataset untuk Demo

In [ ]:
train_data = dataset['train'].select(range(50))
test_data = dataset['test'].select(range(10))

train_data = train_data.shuffle(seed=56)
train_data

Dataset({
    features: ['question', 'answer'],
    num_rows: 50
})

#### d. Preprocess Data

In [ ]:
def format_chat_template(row):
    row_json = [
        {"role": "system", "content": "Doctor AI adalah chatbot kesehatan berbasis AI"},
        {"role": "user", "content": row["question"]},
        {"role": "asistant", "content": row["answer"]},
    ]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

train_data = train_data.map(format_chat_template, num_proc=4)
test_data = test_data.map(format_chat_template, num_proc=4)

train_data['text'][3]

Map (num_proc=4):   0%|          | 0/50 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/10 [00:00<?, ? examples/s]

'<|im_start|>system\nDoctor AI adalah chatbot kesehatan berbasis AI<|im_end|>\n<|im_start|>user\nSiang dok, aku mau tanya. Semalam itu wajah ku nggak kenapa-napa, baik2 aja. Sampai aku pulang sekolah tadi siang. tbtbsaja wajahku full memerah, terasa gatal dan ada bintik-bintik kecil kayak jerawat gitu. Itu kenapa ya dok? terus cara mengatasinya gmana ya dok?<|im_end|>\n<|im_start|>asistant\nAlo, terima kasih atas pertanyaannya untuk Alodokter.Bintik merah kecil yang muncul pada kulit wajah bisa saja disebabkan oleh adanya sunburn akibat paparan sinar matahari, iritasi kulit, calon bakal jerawat seperti bruntusan dan komedo, atau milia yaitu penyumbatan minyak dan kulit mati.<|im_end|>\n'

### 4. Fine-tune Model

#### a. Training Argument

In [ ]:
training_arguments = TrainingArguments(
    output_dir="output",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    warmup_steps=10,
    optim="paged_adamw_8bit",
    num_train_epochs=1,
    eval_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="wandb"
)

#### b. Train

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    peft_config=peft_config,
    max_seq_length=2500,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default 

Step,Training Loss,Validation Loss
3,12.386600,No log
6,10.787000,No log
9,7.661700,No log
12,5.191600,No log


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


TrainOutput(global_step=12, training_loss=8.996911406517029, metrics={'train_runtime': 501.3361, 'train_samples_per_second': 0.1, 'train_steps_per_second': 0.024, 'total_flos': 111566924083200.0, 'train_loss': 8.996911406517029, 'epoch': 0.96})

#### c. Finish Wandb

In [ ]:
wandb.finish()

eval/runtime,▁█▅▂
eval/samples_per_second,█▁▄▇
eval/steps_per_second,█▁▄▇
train/epoch,▁▂▂▂▃▄▄▄▅▅▆▆▇▇███
train/global_step,▁▂▂▂▃▄▄▄▅▅▆▆▇▇███
train/grad_norm,██▇▃▂▂▁▁▂▂▂▁
train/learning_rate,▂▂▃▄▅▅▆▇▇█▅▁
train/loss,▇▇█▆▆▆▄▄▃▂▂▁
eval/runtime,5.2034
eval/samples_per_second,1.922
eval/steps_per_second,1.922


### 5. Test Inference

In [ ]:
messages = [
    {
        "role": "user",
        "content": "Selamat pagi dok, kalo kaki suka keram kenapa ya dok?"
    }
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False,
                                       add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt', padding=True,
                   truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=150,
                         num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])


Selamat pagi dok, karena kaki suka keram karena karena kaki memiliki kemampuan untuk menyerap zat yang ada di dalam tubuh, seperti zat yang ada di dalam makanan yang kita makan, zat yang ada di dalam air, zat yang ada di dalam tubuh kita, dan zat yang ada di dalam tubuh kita.


### 6. Upload Fine-tuned Model ke HF

In [ ]:
output_model = "gemma-2b-chat-doctor-example"
trainer.model.save_pretrained(output_model)
trainer.model.push_to_hub(output_model, use_temp_dir=False)

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


HfHubHTTPError:  (Request ID: Root=1-66f7b681-66080e944774d5eb7cee753c;4061e323-a49e-4241-9d5d-6658c9f139bf)

403 Forbidden: You don't have the rights to create a model under the namespace "rafyardhanie".
Cannot access content at: https://huggingface.co/api/repos/create.
If you are trying to create or update content, make sure you have a token with the `write` role.